In [1]:
import urllib.request as urllib2
import json
import collections, itertools
import requests
import csv
import requests
import shutil
import os
import re
import numpy as np

from bs4 import BeautifulSoup

In [ ]:
# Web scrapping recipes
#import urllib2

def scrap(dish, f, url, total, img=True):
    """Scrapes AllRecipes to get the recipes and images"""
    try:
        # Connect to the URL
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        # Find all recipes on page
        all_items = soup.find_all("div", {'class': 'component card card__recipe card__facetedSearchResult'})

        c = total
        for item in all_items:
            # Find div with the needed information (link to recipe and cover image)
            top_item = item.find("div", {'class': 'card__imageContainer'}
            ).find("a", {'class': 'card__titleLink manual-link-behavior elementFont__title margin-8-bottom'})
            
            dish_url = top_item['href']
            url_title = top_item['title']
            try:
                rating = item.find("div", {"class": "component recipe-ratings"}).find("span", {"class": "review-star-text visually-hidden"}).text
                rating = float(re.findall(r"[-+]?(?:\d*\.\d+|\d+)", rating)[0])
            except:
                rating = np.nan
            
            if img:
                try:
                    # Finding the url to the image
                    img_url_ = top_item.find("div", 
                    {'class': ['component lazy-image lazy-image-udf aspect_1x1',
                              'component lazy-image lazy-image-udf aspect_1x1 cache-only align-default',
                              'component lazy-image lazy-image-udf aspect_1x1 cache-only',
                              'component lazy-image lazy-image-udf aspect_1x1 align-default']}
                    )['data-src']
                except:
                    # If it doesnt work, just move on to next url
                    continue

                try:
                    # Try to save the image as a jpg, if not print error and move on
                    # Might get some HTTPS errors randomly here, maybe timeouts?
                    save_image(dish, img_url_, c)
                except Exception as e:
                    print(e)
                    print(img_url_)
                    continue
                
            # go into the actual recipe link
            page = requests.get(dish_url)
            soup = BeautifulSoup(page.content, 'html.parser')

            #Fetch ingredients
            ingredients = []
            try:
                web_ingredients = soup.find_all("li", {'class': 'ingredients-item'})
                for ing in web_ingredients:
                    # Getting just the ingredient, ignoring the measurement. "Value" seems more standardized,
                    # but less accurate
                    rec_ing = ing.find('input', {'class': 'checkbox-list-input'})['data-ingredient']#['value']
                    rec_quant = ing.find('input', {'class': 'checkbox-list-input'})['data-init-quantity']
                    rec_unit = ing.find('input', {'class': 'checkbox-list-input'})['data-unit']
                    rec_item = ' '.join([rec_quant, rec_unit, rec_ing])
                    ingredients.append(rec_item)
            except Exception as e:
                # If fails, print out why
                print(e)
                print(dish_url)
                print(soup.find("ul", {'class': 'ingredients-section'}))
            
            # Fetch nutrition
            try:
                nutrition = soup.find("div", {"class": "recipeNutritionSectionBlock"}).find("div", {"class": "section-body"}).text.strip().split(";")
            except:
                # If fails, print out why
                print(dish_url)
                print(soup.find("div", {"class": "recipeNutritionSectionBlock"}))

            #Fetch recipe instructions (not really needed but might be)
            recipe_steps = []
            web_recipe = soup.find('ul', {'class': 'instructions-section'})
            if web_recipe is not None:
                web_recipe = web_recipe.find_all('p')
                for rec in web_recipe:
                    recipe_steps.append(rec.text)

            #Write recipe to text file
            result = "|".join([dish, dish_url, dish + "/" + str(c) + ".jpg", f"rating={rating}", str(nutrition), str(ingredients)]) + "\n"
            f.write(result)

            c += 1

    except urllib2.HTTPError:
        return("wrong url")
    return c


def save_image(dish, image_url, num):
    #save image to file 
    resp = requests.get(image_url, stream=True)
    if not os.path.exists("data/" + dish):
        os.makedirs("data/" + dish)
    local_file = open("data/" + dish + "/" + str(num) + ".jpg", 'wb')
    resp.raw.decode_content = True
    shutil.copyfileobj(resp.raw, local_file)
    del resp
    return


def main(page_count):
    #Take food types from classes.txt and search AllRecipes for links
    #Pagecount is how many pages to pull, each page has 25 results I think
    file = open("classes.txt", "r")
    f = open('./data/recipes.txt', 'w', encoding='utf-8')
    all_classes = file.read().split()
    count = 0
    for dish in all_classes:
        total = 0
        count += 1
        print(dish)
        # search the url for recipes
        for i in range(1, page_count+1):
            if dish!='cup_cake':
                url = "https://www.allrecipes.com/search/results/?search=" + '%20'.join(dish.split('_')
                                                )  + "&sort=re" + "&page=" + str(i) 
            else:
                url = "https://www.allrecipes.com/search/results/?search=" + 'cupcake'+ "&sort=re" + "&page=" + str(i) 
            total = scrap(dish, f, url, total, img=True)
    f.close()

In [ ]:
main(page_count = 1)

In [2]:
import os
# os.path.isfile(path)
file1 = open("data/recipes.txt","r")
file2 = open('data/recipes_filtered.txt', 'w')
for i in file1.readlines():
    img_path = i.split('|')[2]
    if os.path.isfile('data/'+img_path):
        file2.write(i)
    else:
        pass
file1.close()
file2.close()

In [3]:
file = open("classes.txt", "r")

all_classes = file.readlines()
class_counts = {}
for i in all_classes:
    j = i.replace('\n', '')
    class_counts[j]=0
pics = open("data/recipes_filtered.txt")
all_pics = pics.readlines()
for i in all_pics:
    clas = i.split('|')[0]
    class_counts[clas]+=1
file.close()
pics.close()

In [4]:
import pandas as pd
count_df = pd.DataFrame({'class':class_counts.keys(), 'count':class_counts.values()})
count_df[count_df['count']<10].sort_values(by='count')

,class,count
17,cheese_plate,0
97,takoyaki,0
3,beef_carpaccio,1
4,beef_tartare,1
86,sashimi,1
7,bibimbap,3
53,hamburger,3
35,escargots,3
38,fish_and_chips,5
28,croque_madame,5
